<a href="https://colab.research.google.com/github/mohamedSalehMatar/AlgaeRecognition./blob/main/AlgaeRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import cv2
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from scipy.fft import fft
from google.colab import files

In [ ]:
# Load the image

print("Upload the template image:")
uploaded = files.upload()
template_filename = list(uploaded.keys())[0]
image = cv2.imdecode(np.frombuffer(uploaded[template_filename], np.uint8), cv2.IMREAD_UNCHANGED)
cv2_imshow(image)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray)

In [ ]:
# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (7, 7), 0)
cv2_imshow(blurred)

# Perform edge detection using Canny
edge = cv2.Canny(blurred, 30, 150)
cv2_imshow(edge)

# Apply morphological closing
kernel = np.ones((5,5),np.uint8)
edges = cv2.morphologyEx(edge, cv2.MORPH_CLOSE, kernel)
cv2_imshow(edges)

In [ ]:
# Find contours in the image
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the filtered contours on the original image
output = image.copy()
cv2.drawContours(output, contours, -1, (0, 255, 0), 2)

cv2_imshow(output)

In [ ]:
# Find the tallest contour
tallest_contour = max(contours, key=lambda contour: cv2.boundingRect(contour)[3])

# Create a blank image of the same size as the original image
signature = np.zeros_like(image)

# Draw the tallest contour on the blank image
cv2.drawContours(signature, [tallest_contour], -1, (255, 255, 255), thickness=cv2.FILLED)

# Find the bounding rectangle of the tallest contour
x, y, w, h = cv2.boundingRect(tallest_contour)

# Crop the signature from the image
signature = signature[y:y+h, x:x+w]

# Now 'signature' contains the extracted signature of the tallest contour
cv2_imshow(signature)

In [ ]:
# Convert the tallest contour points to a 2D array
points = tallest_contour.reshape(-1, 2)

# Separate the x and y coordinates
x = points[:, 0]
y = points[:, 1]

# Plot the points of the tallest contour
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(x, y)
plt.title('Contour Plot')

# Calculate the center of the contour
cx = np.mean(x)
cy = np.mean(y)

# Calculate the distance and angle for each point in the contour
distances = np.sqrt((x-cx)**2 + (y-cy)**2)
angles = np.arctan2(y-cy, x-cx)

# Plot distance versus angle
plt.subplot(1,2,2)
plt.plot(np.degrees(angles), distances)
plt.title('Distance vs Angle')
plt.xlabel('Angle (degrees)')
plt.ylabel('Distance')

# Show the plots
plt.tight_layout()
plt.show()